# Benchmarking Thinc layers with a custom `benchmark` layer

This notebook shows how to write a `benchmark` layer that can wrap any layer(s) in your network and that **logs the execution times** of the initialization, forward pass and backward pass. The benchmak layer can also be mapped to an operator like `@` to make it easy to add debugging to your network.

In [ ]:
!pip install thinc

To log the results, we first set up a custom logger using Python's `logging` module. You could also just print the stats instead, but using `logging` is cleaner, since it lets other users modify the logger's behavior more easily, and lets separates the logs from other output and write it to a file (e.g. if you're benchmarking several layers during training). The following logging config will output the date and time, the name of the logger and the logged results.

In [2]:
import logging

logger = logging.getLogger("thinc:benchmark")
if not logger.hasHandlers():  # prevent Jupyter from adding multiple loggers
    formatter = logging.Formatter('%(asctime)s %(name)s  %(message)s', datefmt="%Y-%m-%d %H:%M:%S")
    handler = logging.StreamHandler()
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    logger.setLevel(logging.DEBUG)

Here's a minimalistic time logger that can be initialized with the name of a given layer, and can track several events (e.g. `"forward"` and `"backward"`). When the `TimeLogger.end` method is called, the output is formatted nicely and the elapsed time is logged with the logger name and colored label.

In [3]:
from timeit import default_timer
from wasabi import color


class TimeLogger:
    def __init__(self, name):
        self.colors = {"forward": "green", "backward": "blue"}
        self.name = name
        self.timers = {}
        
    def start(self, name):
        self.timers[name] = default_timer()
        
    def end(self, name):
        result = default_timer() - self.timers[name]
        label = f"{name.upper():<8}"
        label = color(label, self.colors.get(name), bold=True)
        logger.debug(f"{self.name:<12} | {label} | {result:.6f}")

The `benchmark` layer now has to wrap the forward pass, backward pass and initialization of the layer it wraps and log the execution times. It then returns a Thinc model instance with the custom `forward` function and a custom `init` function. We'll also allow setting a custom `name` to make it easier to tell multiple wrapped benchmark layers apart.

In [4]:
from thinc.api import Model
        
def benchmark(layer, name=None):
    name = name if name is not None else layer.name
    t = TimeLogger(name)
    
    def init(model, X, Y):
        t.start("init")
        result = layer.initialize(X, Y)
        t.end("init")
        return result
        
    def forward(model, X, is_train):
        t.start("forward")
        layer_Y, layer_callback = layer(X, is_train=is_train)
        t.end("forward")
        
        def backprop(dY):
            t.start("backward")
            result = layer_callback(dY)
            t.end("backward")
            return result
        
        return layer_Y, backprop
        
    return Model(f"benchmark:{layer.name}", forward, init=init) 

---

## Usage examples

### Using the `benchmark` layer as a function

We can now wrap one or more layers (including nested layers) with the `benchmark` function. This is the original model:

```python
model = chain(Linear(1), Linear(1))
```

In [6]:
import numpy
from thinc.api import chain, Linear

X = numpy.zeros((1, 2), dtype="f")

model = benchmark(chain(benchmark(Linear(1)), Linear(1)), name="outer")
model.initialize(X=X)
Y, backprop = model(X, is_train=False)
dX = backprop(Y)

2020-01-21 19:14:10 thinc:benchmark  linear       | INIT     | 0.000337
2020-01-21 19:14:10 thinc:benchmark  linear       | FORWARD  | 0.000497
2020-01-21 19:14:10 thinc:benchmark  outer        | INIT     | 0.003808
2020-01-21 19:14:10 thinc:benchmark  linear       | FORWARD  | 0.000139
2020-01-21 19:14:10 thinc:benchmark  outer        | FORWARD  | 0.001537
2020-01-21 19:14:10 thinc:benchmark  linear       | BACKWARD | 0.000069
2020-01-21 19:14:10 thinc:benchmark  outer        | BACKWARD | 0.001613


### Using the `benchmark` layer as an operator

Alternatively, we can also use `Model.define_operators` to map `benchmark` to an operator like `@`. The left argument of the operator is the first agument passed into the function (the layer) and the right argument is the second argument (the name). The following example wraps the whole network (two chained `Linear` layers) in a benchmark layer named `"outer"`, and the first `Linear` layer in a benchmark layern named `"first"`.

In [7]:
from thinc.api import Model

with Model.define_operators({">>": chain, "@": benchmark}):
    model = (Linear(1) @ "first" >> Linear(1)) @ "outer"
    
model.initialize(X=X)
Y, backprop = model(X, is_train=True)
dX = backprop(Y)

2020-01-21 19:14:13 thinc:benchmark  first        | INIT     | 0.000298
2020-01-21 19:14:13 thinc:benchmark  first        | FORWARD  | 0.000235
2020-01-21 19:14:13 thinc:benchmark  outer        | INIT     | 0.005356
2020-01-21 19:14:13 thinc:benchmark  first        | FORWARD  | 0.000172
2020-01-21 19:14:13 thinc:benchmark  outer        | FORWARD  | 0.006204
2020-01-21 19:14:13 thinc:benchmark  first        | BACKWARD | 0.000097
2020-01-21 19:14:13 thinc:benchmark  outer        | BACKWARD | 0.001783


### Using the `benchmark` layer during training

In [10]:
from thinc.api import Model, chain, ReLu, Softmax, Adam, evaluate_model_on_arrays, minibatch
import ml_datasets

n_hidden = 32
dropout = 0.2

with Model.define_operators({">>": chain, "@": benchmark}):
    model = (
        ReLu(nO=n_hidden, dropout=dropout) @ "relu1"
        >> ReLu(nO=n_hidden, dropout=dropout) @ "relu2"
        >> Softmax()
    )

X = numpy.zeros((5, 784), dtype="f")
Y = numpy.zeros((540, 10), dtype="f")

model.initialize(X=X[:5], Y=Y[:5])
optimizer = Adam(0.001)
indices = model.ops.xp.arange(X.shape[0], dtype="i")
for i in range(10):
    model.ops.xp.random.shuffle(indices)
    for idx_batch in minibatch(indices):
        Yh, backprop = model.begin_update(X[idx_batch])
        backprop(Yh - Y[idx_batch])
        model.finish_update(optimizer)

2020-01-21 19:14:50 thinc:benchmark  relu1        | INIT     | 0.000903
2020-01-21 19:14:50 thinc:benchmark  relu1        | FORWARD  | 0.000236
2020-01-21 19:14:50 thinc:benchmark  relu2        | INIT     | 0.001982
2020-01-21 19:14:50 thinc:benchmark  relu2        | FORWARD  | 0.000525
2020-01-21 19:14:50 thinc:benchmark  relu1        | FORWARD  | 0.000717
2020-01-21 19:14:50 thinc:benchmark  relu2        | FORWARD  | 0.000187
2020-01-21 19:14:50 thinc:benchmark  relu1        | FORWARD  | 0.000574
2020-01-21 19:14:50 thinc:benchmark  relu2        | FORWARD  | 0.000234
2020-01-21 19:14:50 thinc:benchmark  relu2        | BACKWARD | 0.000153
2020-01-21 19:14:50 thinc:benchmark  relu1        | BACKWARD | 0.000346
2020-01-21 19:14:50 thinc:benchmark  relu1        | FORWARD  | 0.000314
2020-01-21 19:14:50 thinc:benchmark  relu2        | FORWARD  | 0.000332
2020-01-21 19:14:50 thinc:benchmark  relu2        | BACKWARD | 0.000095
2020-01-21 19:14:50 thinc:benchmark  relu1        | BACKWARD | 0